In [1]:
import sympy as sm

#turn on pretty printing
sm.init_printing()

# Define the symbols
l = sm.symbols('l')

y = 2*(sm.sqrt(l**2 + 1)-l)

# Print the result
sm.simplify(y)

In [2]:
import chebpy as cp
import numpy as np



d1 = cp.chebfun('d', [0, 1])

l1=1.4
l2=0.4
h1=0.5
h2=0.1
k_theta=1
k_phi=1
k_L=6000


L1 = np.sqrt(l1**2 + h1**2)
th0 = np.arctan(h1/l1)
th = np.arctan((h1-d1)/l1)

d2 = 2*(np.sqrt(l1**2+h1**2-(h1-d1)**2)-l1)
ph0 = np.arctan(h2/l2)
ph = np.arctan((h2-d2)/l2)

d3 = np.sqrt(l2**2+h2**2-(h2-d2)**2)-l2

L2 = np.sqrt(l2**2+h2**2)

PE1 = 1/2*k_theta*(th0-th)**2
PE2 = 1/2*k_phi*(ph0-ph)**2
PE3 = 1/2*k_L*(d3)**2

PE = 8*PE1+8*PE2+2*PE3




In [3]:
import chebpy as cp
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from matplotlib.gridspec import GridSpec
from spring import spring


prev_eq = 0
def plot_chebpy(l1, l2, alpha, k_theta, k_phi, k_L, d, F_buckle, K_extension1, K_extension2, K_extension3, input_displacements_option):
    global prev_eq
    K_extensions = [K_extension1, K_extension2, K_extension3]
    input_displacements_map = {
        "All 0": [0, 0, 0],
        "(-1, -1, -1)": [-1, -1, -1],
        "(-1, -1, 1)": [-1, -1, 1],
        "(-1, 1, -1)": [-1, 1, -1],
        "(-1, 1, 1)": [-1, 1, 1],
        "(1, -1, -1)": [1, -1, -1],
        "(1, -1, 1)": [1, -1, 1],
        "(1, 1, -1)": [1, 1, -1],
        "(1, 1, 1)": [1, 1, 1],
    }
    input_displacements = input_displacements_map[input_displacements_option]
    # Unpack spring parameters


    d1 = cp.chebfun('d', [-0.1, 1.1])
    h1 = 0.5 #fixed parameter

    L1 = np.sqrt(l1**2 + h1**2)
    th0 = np.arctan(h1/l1)
    th = np.arctan((h1-d1)/l1)
    

    h2 = alpha*(np.sqrt(l1**2+h1**2)-l1) #fixed parameter
    d2 = 2*(np.sqrt(l1**2+h1**2-(h1-d1)**2)-l1)
    ph0 = np.arctan(h2/l2)
    ph = np.arctan((h2-d2)/l2)

    d3 = np.sqrt(l2**2+h2**2-(h2-d2)**2)-l2
    d3_val = d3(d)

    L2 = np.sqrt(l2**2+h2**2)

    PE1 = 1/2*k_theta*(th0-th)**2
    PE2 = 1/2*k_phi*(ph0-ph)**2
    PE3 = 1/2*k_L*(d3)**2

    PE = 8*PE1+8*PE2+2*PE3

    F = PE.diff()
    dF = F.diff()
    spring_forces = [(K_extensions[i] * (input_displacements[i] - d1).maximum(-F_buckle)) for i in range(3)]
    #spring_forces = [(K_extensions[i] * (input_displacements[i] - d1)) for i in range(3)]
    # Update the force balance to include the springs
    net_force = F-sum(spring_forces)
    r = F.roots()
    net_r = net_force.roots()

    stabler = r[dF(r) > 0]
    net_r = net_r[dF(net_r) > 0]
    #r = F.roots()

    extrema = dF.roots()

    # Mechanism plot
    #d is chosen from net_r as the root closest to the prev eq using min(abs())
    if prev_eq == 0:
        d = net_r[0]
    else:
        d = net_r[np.argmin(abs(net_r-prev_eq))]
    prev_eq = d
    
    #d=net_r[0]
    th_val = th(d)

    c = np.cos(th_val)
    ys = [0, L1*c, 2*L1*c, 2*l1+h2, 2*L1*c, 2*l1+h2]
    xs = [0, -h1+d, 0, l2+d3_val, 0, -l2-d3_val]

    # plot springs
    [sp1x,sp1y] = spring(d-h1,1.25*l1,4*h1+input_displacements[0],1.25*l1,8,5*h1,h1/5)
    [sp2x,sp2y] = spring(d-h1,1.00*l1,4*h1+input_displacements[1],1.00*l1,8,5*h1,h1/5)
    [sp3x,sp3y] = spring(d-h1,0.75*l1,4*h1+input_displacements[2],0.75*l1,8,5*h1,h1/5)



    fig = plt.figure(figsize=(12, 6))
    gs = GridSpec(2, 2, figure=fig)

    ax1 = fig.add_subplot(gs[0, 0])
    
    ax2 = fig.add_subplot(gs[:, 1])
    ax3 = fig.add_subplot(gs[1, 0])

    
    plt.subplots_adjust(hspace=0.3)

    # First subplot
    F.plot(ax=ax1,label="Mechanism Force")
    net_force.plot(ax=ax1,label="Net Force")
    ax1.plot(net_r, net_force(net_r), '.r', markersize=13)
    ax1.plot(stabler, F(stabler), '.g', markersize=10)
    #ax1.plot(extrema, F(extrema), '.k', markersize=10)
    ax1.axvline(d, color='g', linestyle='--')
    sum(spring_forces).plot(ax=ax1,label="Spring Forces")
    ax1.legend()
    ax1.set_title('Force Diplacement Curve')
    ax1.set_xlabel('Displacement')
    ax1.set_ylabel('Force')
    ax1.set_ylim([-(K_extension1+K_extension2+K_extension3), (K_extension1+K_extension2+K_extension3)])
    ax1.grid(True)

    # Second subplot
    ax2.plot(xs, ys, '-o')
    ax2.set_aspect('equal', 'box')
    ax2.set_title('Mechanism')
    #plot springs
    ax2.plot(sp1x,sp1y,'k',linewidth=1)
    ax2.plot(sp2x,sp2y,'k',linewidth=2)
    ax2.plot(sp3x,sp3y,'k',linewidth=2)
    ax2.set_xlim([-h1*1.5, 2*l1+h2+0.5])
    ax2.set_ylim([0,2*L1+0.5 ])
    ax2.axvline(2*h1, linestyle='--')
    ax2.axvline(4*h1, linestyle='--')
    ax2.axvline(6*h1, linestyle='--')
    


    # New subplot (Potential Energy Components)
    #ax3.set_title('Potential Energy Components')
    ax3.set_xlabel('Displacement')
    ax3.set_ylabel('Potential Energy')

    (8 * PE1).plot(ax=ax3, label="PE1 (Theta)")
    (8 * PE2).plot(ax=ax3, label="PE2 (Phi)")
    (2 * PE3).plot(ax=ax3, label="PE3 (L)")
    (PE).plot(ax=ax3, label="Total PE")
    ax3.axvline(d, color='g', linestyle='--')
    ax3.legend()
    



    

# Define slider widgets for parameters
F_buckle_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.05, description=f'F_buckle:')
K_extension_sliders = [widgets.FloatSlider(min=0.1, max=30, step=1, value=10, description=f'K_extension{i+1}:') for i in range(3)]
input_displacement_sliders = [widgets.FloatSlider(min=-1, max=1, step=0.1, value=0, description=f'input_disp{i+1}:') for i in range(3)]
K_extension_sliders[0].value = 8
K_extension_sliders[1].value = 12
K_extension_sliders[2].value = 12

#Define input states radio buttons
input_displacement_radio = widgets.RadioButtons(
    options=[
        "All 0",
        "(-1, -1, -1)",
        "(-1, -1, 1)",
        "(-1, 1, -1)",
        "(-1, 1, 1)",
        "(1, -1, -1)",
        "(1, -1, 1)",
        "(1, 1, -1)",
        "(1, 1, 1)",
    ],
    description="Input Displacements:",
    style={"description_width": "initial"},
)


# Define slider widgets for parameters
l1_slider = widgets.FloatSlider(min=0.7, max=2.8, step=0.1, value=1.4, description='l1:')
l2_slider = widgets.FloatSlider(min=0.1, max=1, step=0.1, value=0.4, description='l2:')
alpha_slider = widgets.FloatSlider(min=0.8, max=1.4, step=0.1, value=1.1, description='alpha:')
k_theta_slider = widgets.FloatSlider(min=0.1, max=10, step=0.1, value=1, description='k_theta:')
k_phi_slider = widgets.FloatSlider(min=0.1, max=10, step=0.1, value=0.1, description='k_phi:')
k_L_slider = widgets.FloatSlider(min=3500, max=10000, step=100, value=7000, description='k_L:')
d_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0, description='d:')

# Create an interactive plot with the sliders
out = widgets.interactive_output(plot_chebpy, {
    'l1': l1_slider,
    'l2': l2_slider,
    'alpha': alpha_slider,
    'k_theta': k_theta_slider,
    'k_phi': k_phi_slider,
    'k_L': k_L_slider,
    'd': d_slider,
    'F_buckle': F_buckle_slider,
    'K_extension1': K_extension_sliders[0],
    'K_extension2': K_extension_sliders[1],
    'K_extension3': K_extension_sliders[2],
    'input_displacements_option': input_displacement_radio
})



slider_box1 = widgets.VBox([l1_slider, l2_slider, alpha_slider])
slider_box2 = widgets.VBox([k_theta_slider, k_phi_slider, k_L_slider, d_slider])
slider_box3 = widgets.VBox([F_buckle_slider]+ K_extension_sliders)
slider_box4 = widgets.VBox([input_displacement_radio])
slider_layout = widgets.HBox([slider_box1, slider_box2, slider_box3, slider_box4])

# Display the custom layout and the output of the interactive plot using widgets.VBox
display(widgets.VBox([slider_layout,out]))





In [11]:
#make a new plot with just the force displacement curve
import pandas as pd
dfs=[]
file = "data/Response Graph-6.csv"
df = pd.read_csv(file,skiprows=9)
df = df.iloc[:,0:2]
df.columns = ['Time','Response']
dfs.append(df)

file = "data/Response Graph-7.csv"
def plot_forces(l1, l2, alpha, k_theta, k_phi, k_L, d):
    

    input_displacements_map = {
        "All 0": [0, 0, 0],
        "(-1, -1, -1)": [-1, -1, -1],
        "(-1, -1, 1)": [-1, -1, 1],
        "(-1, 1, -1)": [-1, 1, -1],
        "(-1, 1, 1)": [-1, 1, 1],
        "(1, -1, -1)": [1, -1, -1],
        "(1, -1, 1)": [1, -1, 1],
        "(1, 1, -1)": [1, 1, -1],
        "(1, 1, 1)": [1, 1, 1],
    }
    # Unpack spring parameters


    d1 = cp.chebfun('d', [0, 1])
    h1 = 0.5 #fixed parameter

    L1 = np.sqrt(l1**2 + h1**2)
    th0 = np.arctan(h1/l1)
    th = np.arctan((h1-d1)/l1)
    

    h2 = alpha*(np.sqrt(l1**2+h1**2)-l1) #fixed parameter
    d2 = 2*(np.sqrt(l1**2+h1**2-(h1-d1)**2)-l1)
    ph0 = np.arctan(h2/l2)
    ph = np.arctan((h2-d2)/l2)

    d3 = np.sqrt(l2**2+h2**2-(h2-d2)**2)-l2
    d3_val = d3(d)

    L2 = np.sqrt(l2**2+h2**2)

    PE1 = 1/2*k_theta*(th0-th)**2
    PE2 = 1/2*k_phi*(ph0-ph)**2
    PE3 = 1/2*k_L*(d3)**2

    PE = 8*PE1+8*PE2+2*PE3

    F = PE.diff()
    F1 = 8*PE1.diff()
    F2 = 8*PE2.diff()
    F3 = 2*PE3.diff()
    dF = F.diff()
    #spring_forces = [(K_extensions[i] * (input_displacements[i] - d1)) for i in range(3)]
    # Update the force balance to include the springs


    fig = plt.figure(figsize=(12, 6))
    ax1 = fig.add_subplot(1, 1, 1)
    ax1.plot(df['Time'],df['Response'],label="FEA")
    F.plot(ax=ax1,label="Mechanism Force")
    # F1.plot(ax=ax1,label="Force 1")
    # F2.plot(ax=ax1,label="Force 2")
    # F3.plot(ax=ax1,label="Force 3")
    ax1.set_xlabel('d1')
    ax1.set_ylabel('Force')
    ax1.set_title('Force vs. d1')
    ax1.legend()
    ax1.grid(True)
    plt.show()
    
    

# Create sliders for the spring parameters
# Define slider widgets for parameters
l1_slider = widgets.FloatSlider(min=0.7, max=2.8, step=0.1, value=1.4, description='l1:')
l2_slider = widgets.FloatSlider(min=0.1, max=1, step=0.1, value=0.4, description='l2:')
alpha_slider = widgets.FloatSlider(min=0.8, max=1.4, step=0.01, value=1.1, description='alpha:')
k_theta_slider = widgets.FloatSlider(min=0.1, max=20, step=0.1, value=1, description='k_theta:')
k_phi_slider = widgets.FloatSlider(min=0.1, max=20, step=0.1, value=0.1, description='k_phi:')
k_L_slider = widgets.FloatSlider(min=3500, max=40000, step=100, value=7000, description='k_L:')
d_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0, description='d:')

# Create an interactive plot with the sliders
out = widgets.interactive_output(plot_forces, {
    'l1': l1_slider,
    'l2': l2_slider,
    'alpha': alpha_slider,
    'k_theta': k_theta_slider,
    'k_phi': k_phi_slider,
    'k_L': k_L_slider,
    'd': d_slider
})

# Create a custom layout for the sliders
slider_box1 = widgets.VBox([l1_slider, l2_slider, alpha_slider])
slider_box2 = widgets.VBox([k_theta_slider, k_phi_slider, k_L_slider, d_slider])

# Display the custom layout and the output of the interactive plot using widgets.VBox
display(widgets.VBox([slider_box1, slider_box2, out]))


